In [1]:
import csv
import pandas as pd
import numpy as np
import nvector as nv

import folium
import folium.plugins as plugins
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display

#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

#create a dictionary of the army groups and nums to use in a selection widget
army_groups = dict()
for group_keys in grouped_div.groups.keys():
    army_group = group_keys[0]
    div_num = group_keys[1]
    if isinstance(div_num,str):
        if army_group in army_groups:
            army_groups[army_group].append(div_num)
        else:
            army_groups[army_group] = ['-',div_num]

#output widget for the map
out = widgets.Output()

#selection widgets for army group and num
group_sel = widgets.Dropdown(options=['-']+list(army_groups.keys()))
div_sel = widgets.Dropdown(options=['-'])

#event handler
def on_group_sel(change):
    if change['new'] in army_groups:
        div_sel.options = army_groups[change['new']]
        plot_div_movement(change['new'],army_groups[change['new']][0])
    if change['new'] == '-':
        div_sel.options = ['-']
        plot_div_movement(change['new'],'-')

group_sel.observe(on_group_sel,'value')

def on_div_sel(change):
    if change['new'] is not None:
        plot_div_movement(group_sel.value,change['new'])
    
div_sel.observe(on_div_sel,'value')

#get the origin and destination of a division
def get_div_origin_dest(div_data_df):
    div_data = dict()
    for row in div_data_df.itertuples():
        if row.MAP_DATE not in div_data:
            div_data[row.MAP_DATE] = dict()
            div_data[row.MAP_DATE]['POINT_Y'] = row.POINT_Y
            div_data[row.MAP_DATE]['POINT_X'] = row.POINT_X
    div_locs = []
    for key in sorted(div_data):
        div_loc = dict()
        div_loc['POINT_Y'] = div_data[key]['POINT_Y']
        div_loc['POINT_X'] = div_data[key]['POINT_X']
        div_locs.append(div_loc)
    #there is an origin and dest
    if len(div_locs) > 1:
        origin = div_locs[0]
        dest = div_locs[len(div_locs)-1]
        return [[origin['POINT_Y'],origin['POINT_X']],
                [dest['POINT_Y'],dest['POINT_X']]]
    else:
        return None

#gets the vector's bearing from North
def get_bearing(orig, dest):
    wgs84 = nv.FrameE(name='WGS84')
    pointA = wgs84.GeoPoint(latitude=orig[0], longitude=orig[1], degrees=True)
    pointB = wgs84.GeoPoint(latitude=dest[0], longitude=dest[1], degrees=True)
    p_AB_E = nv.diff_positions(pointA, pointB)
    frame_N = nv.FrameN(pointA)
    p_AB_N = p_AB_E.change_frame(frame_N)
    p_AB_N = p_AB_N.pvector.ravel()
    azimuth = np.arctan2(p_AB_N[1], p_AB_N[0])
    return np.rad2deg(azimuth)

#adds a division's movement vector to the map
def add_div_vect_to_map(dest_map,locations):
    #first the position markers
    orig = locations[0]
    dest = locations[1]
    folium.CircleMarker(orig,radius=4,color='green').add_to(dest_map)
    folium.CircleMarker(dest,radius=4,color='red').add_to(dest_map)
    vect_pair = [orig,dest]
    vect_line = folium.PolyLine(locations=vect_pair,weight=1,color='black')
    dest_map.add_child(vect_line)
    #get vector direction
    rotation = get_bearing(orig,dest) - 90 #to account for eastward initial rotation
    #now display arrows in the direction
    arrow_lats = np.linspace(orig[0], dest[0], 4)[1:3]
    arrow_lons = np.linspace(orig[1], dest[1], 4)[1:3]
    for points in zip(arrow_lats, arrow_lons):
        folium.RegularPolygonMarker(location=points, 
                                    fill_color='black', number_of_sides=3, 
                                    radius=4, rotation=rotation).add_to(dest_map)
    
#main function to plot the movement of a selected group and num
def plot_div_movement(army_group,num_name):
    division_map = folium.Map([54.78, 32.04],zoom_start=6)
    if army_group == '-':
        #plot all division vectors
        for group_key in grouped_div.groups.keys():
            #weed out groups that don't have a valid num
            if isinstance(group_key[1],str):
                div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','MAP_DATE']]
                coordinates = get_div_origin_dest(div_df)
                #only if there is movement
                if coordinates is not None:
                    add_div_vect_to_map(division_map,coordinates)
    else:
        #specific army group and num
        #default is to display all division nums for this group
        if num_name == '-':
            for group_key in grouped_div.groups.keys():
                #weed out groups that don't have a valid num and groups that are not the selected one
                if isinstance(group_key[1],str) and group_key[0] == army_group:
                    div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','MAP_DATE']]
                    coordinates = get_div_origin_dest(div_df)
                    #only if there is movement
                    if coordinates is not None:
                        add_div_vect_to_map(division_map,coordinates)
        else:
            group_key = (army_group,num_name)
            div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y','MAP_DATE']]
            coordinates = get_div_origin_dest(div_df)
            #only if there is movement
            if coordinates is not None:
                add_div_vect_to_map(division_map,coordinates)
            
    out.clear_output()
    with out:
        iframe = division_map._repr_html_()
        display(HTML(iframe))
    return

#output UI
sel_ui = widgets.HBox([group_sel,div_sel])
res = widgets.VBox([sel_ui,out])
plot_div_movement('-','-')
res

In [3]:
#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])


In [4]:
df

,OBJECTID,Army_Group,Num_Name,MAP_DATE,Situation,Type_D,Weight_D,Weight_ND,Type_ND,CU_KIND,Fraction_Division,ORIG_FID,Fraction_Div,Label,POINT_X,POINT_Y
0,1,Brigade,900,1941-07-24,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,1.0,1.00,900,32.658135,55.407984
1,2,Brigade,900,1941-07-17,1.0,NaN,NaN,2.0,MOT BDE,0,NaN,2.0,1.00,900,29.921727,55.468670
2,3,Brigade,900,1941-08-01,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,3.0,1.00,900,32.791192,55.390543
3,4,Brigade,900,1941-08-18,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,4.0,1.00,900,32.698244,55.278816
4,5,Brigade,900,1941-07-13,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,5.0,1.00,900,29.784361,55.092483
5,6,Brigade,900,1941-08-24,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,6.0,1.00,900,32.708930,55.251919
6,7,Brigade,900,1941-08-20,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,7.0,1.00,900,32.777420,55.246717
7,8,Brigade,900,1941-08-23,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,8.0,1.00,900,32.717698,55.247718
8,9,Brigade,900,1941-08-02,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,9.0,1.00,900,32.744523,55.364359
9,10,Brigade,900,1941-07-12,0.0,NaN,NaN,2.0,MOT BDE,0,NaN,10.0,1.00,900,29.843918,54.953984


In [2]:
grouped_div = df.groupby(['Army_Group','Num_Name'])

In [3]:
grouped_div.groups

{('A.A.', nan): Int64Index([771], dtype='int64'),
 ('A.A.', '321'): Int64Index([675], dtype='int64'),
 ('A.A.', '53'): Int64Index([1536], dtype='int64'),
 ('A.A.Panzer Division', '7'): Int64Index([123, 372], dtype='int64'),
 ('A.V.', '19'): Int64Index([108], dtype='int64'),
 ('Brigade',
  '900'): Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
               11,   12,   13,  193,  203,  222,  247,  276,  289,  302,  486,
             1992, 2049, 2072, 2200, 2201, 2202, 2203, 2204, 2205],
            dtype='int64'),
 ('CAV Division',
  '1'): Int64Index([  71,   92,  168,  183,  250,  268,  354,  377,  391,  407,  513,
             1092, 1698, 2009, 2034, 2046, 2082, 2087, 2126, 2129],
            dtype='int64'),
 ('Gr. Panzer Division', '2'): Int64Index([722], dtype='int64'),
 ('Gr. Schubert', nan): Int64Index([1682], dtype='int64'),
 ('Gr.Behlendorff', nan): Int64Index([1685, 1686, 1687], dtype='int64'),
 ('Gr.Schubert', nan): Int64Index([1677], dtype='int